# ATMS 523 Module 3 Homework
Megan Walker-Radtke

## Instructions
1. Adapt the code from class that reads GHCN Daily Data from Amazon Web Services and write a function that will download the station you want (called with a GHCN station ID), and calculate the all time record high and low and the normal (mean) high and low temperature for the 1991-2020 period for the desired station and returns a pandas data frame with the columns ['record_min_temp', average_min_temp', 'average_max_temp', record_max_temp'].  Write a code that can call this function and successfully demonstrate that it works.

2. Develop a plot (using matplotlib) that displays for the city of choice a plot showing the record, average, and actual high and low temperatures for that year and city.  
     
     You are permitted to use the "weather" example from the `bokeh` gallery as inspiration.  A running example for what the plot could look like is here: [Weather](https://demo.bokeh.org/weather), and the GitHub repository for the dashboard is [here](https://github.com/bokeh/bokeh/tree/branch-3.9/examples/server/app/weather). Note that you do not have to use bokeh for this assignment, you can use matplotlib!

## Approach
### Write a function to calculate the station climatology
1. Download GHCN Daily Data from AWS S3 for a station of interest using adaptation of code presented in class
2. NaN check:
     - vars: tmax, tmin
     - print: total number of NaNs
     - plot: timeline showing dates with NaNs (purpose: ID whether stations has large data gaps)
3. Calculate climatology:
     - Pull out subset of data for 1991 - 2020
     - Groupby day of the year (e.g. all Jan1 together, all Jan2 together, etc.)
     - Calculate extremes for each day of the year 
          - all-time tmax
          - all-time tmin
     - Calculate means for each day of the year
          - mean tmax
          - mean tmin
4. Save the station climatology to Pandas DF with rows (index) being day of the year and columns ['record_min_temp', average_min_temp', 'average_max_temp', record_max_temp']

### Create a plot that compares daily temperatures in a given year to the 1991-2020 climatology
Plot elements:
1. Climatology
     - Shaded range of extremes (abs tmax: abs tmin)
     - Shaded range of means (mean tmax: mean tmin)
2. Temperatures for a specified year
     - shaded range of tmax: tmin for that year

Plot specifications:
- Horizontal axis: day of the year
- Vertical axis: temperature (deg C)

In [1]:
# Import libraries

import pandas as pd, numpy as np
from pathlib import Path
import fsspec
import matplotlib.pyplot as plt
import seaborn as sns

### Inventory Available Data

In [13]:
# Set up to bring in GHCN data from AWS S3
 
S3_STATIONS_TXT   = "s3://noaa-ghcn-pds/ghcnd-stations.txt"
S3_INVENTORY_TXT  = "s3://noaa-ghcn-pds/ghcnd-inventory.txt"
S3_BY_STATION     = "s3://noaa-ghcn-pds/csv/by_station/{id}.csv"
STOR = {"anon": True}

OUTDIR = Path('../data'); OUTDIR.mkdir(parents=True, exist_ok=True)
OUT_PARQUET = OUTDIR / f'ghcn_{stationID}.parquet'
OUT_CSV = OUTDIR / f'ghcn_{stationID}.csv'
print('Output:', OUT_PARQUET.resolve())

Output: /Users/meganwalker/MDW Docs/Academic/Univ Illinois/ATMS 523/module3/data/ghcn_USW00012816.parquet


In [3]:
# Load GHCN data from AWS S3 and check out what's available

colspecs = [(0,11),(12,20),(21,30),(31,37),(38,40),(41,71),(72,75),(76,79),(80,85)]
names = ['ID','LATITUDE','LONGITUDE','ELEVATION','STATE','NAME','GSN_FLAG','HCN_CRN_FLAG','WMO_ID']

stations = pd.read_fwf(S3_STATIONS_TXT, colspecs=colspecs, names=names, dtype={'ID':str,'STATE':str,'WMO_ID':str}, storage_options=STOR)
stations['NAME'] = stations['NAME'].str.strip(); stations['STATE'] = stations['STATE'].fillna('').str.strip()

inventory = pd.read_csv(
    S3_INVENTORY_TXT, sep=r'\s+', names=['ID','LAT','LON','ELEMENT','FIRSTYEAR','LASTYEAR'],
    dtype={'ID':str,'ELEMENT':str,'FIRSTYEAR':int,'LASTYEAR':int}, engine='python', storage_options=STOR
)

stations.head(), inventory.head()

(            ID  LATITUDE  LONGITUDE  ELEVATION STATE                   NAME  \
 0  ACW00011604   17.1167   -61.7833       10.1        ST JOHNS COOLIDGE FLD   
 1  ACW00011647   17.1333   -61.7833       19.2                     ST JOHNS   
 2  AE000041196   25.3330    55.5170       34.0          SHARJAH INTER. AIRP   
 3  AEM00041194   25.2550    55.3640       10.4                   DUBAI INTL   
 4  AEM00041217   24.4330    54.6510       26.8               ABU DHABI INTL   
 
   GSN_FLAG HCN_CRN_FLAG WMO_ID  
 0      NaN          NaN    NaN  
 1      NaN          NaN    NaN  
 2      GSN          NaN  41196  
 3      NaN          NaN  41194  
 4      NaN          NaN  41217  ,
             ID      LAT      LON ELEMENT  FIRSTYEAR  LASTYEAR
 0  ACW00011604  17.1167 -61.7833    TMAX       1949      1949
 1  ACW00011604  17.1167 -61.7833    TMIN       1949      1949
 2  ACW00011604  17.1167 -61.7833    PRCP       1949      1949
 3  ACW00011604  17.1167 -61.7833    SNOW       1949      194

### Gather data of interest

In [4]:
# Get user input for state and city of interest

state = input("Enter 2-letter state code (e.g., 'FL'): ").strip().upper() # remove accidental whitespace and force the state abbreviation to uppercase
city = input("Enter city name (e.g., 'Gainesville'): ").strip().upper() # remove accidental whitespace and force the city name to uppercase to match GHCN format


In [6]:
# Narrow down to stations in state of interest with at least 30 years of data, with the last year of 2021 or later (so that climatology can be computed)

# Calculate period of record for each station
coverage = (inventory.groupby('ID', as_index=False)
                    .agg(first=('FIRSTYEAR','min'), last=('LASTYEAR','max'))
                    .assign(years=lambda d: d['last'] - d['first'] + 1))

state_list = (stations.loc[stations['STATE']==state, ['ID','NAME','STATE','LATITUDE','LONGITUDE','ELEVATION']]
              .merge(coverage, on='ID', how='inner'))
state30 = state_list[state_list['years']>=30].copy()

# Keep only those stations that have coverage through 2021 or later
state30_clim = state30[state30['last'] >= 2021].copy()
state30_clim.sort_values(['years','ID'], ascending=[False, True]).head(12)

,ID,NAME,STATE,LATITUDE,LONGITUDE,ELEVATION,first,last,years
1826,USC00080478,BARTOW 1SE,FL,27.8864,-81.8325,34.1,1892,2025,134
1891,USC00082229,DELAND 1 SSE,FL,29.0097,-81.2986,7.6,1892,2025,134
1903,USC00082944,FERNANDINA BEACH,FL,30.6669,-81.4525,14.9,1892,2025,134
1958,USC00084731,LAKE CITY 2 E,FL,30.1853,-82.5942,59.4,1892,2025,134
2021,USC00086414,OCALA,FL,29.1639,-82.0778,22.9,1892,2025,134
2060,USC00087205,PLANT CITY,FL,28.0208,-82.1392,33.2,1892,2025,134
2116,USC00088824,TARPON SPGS SEWAGE PLT,FL,28.1522,-82.7539,1.8,1892,2025,134
2179,USW00012835,FT MYERS PAGE FLD AP,FL,26.5850,-81.8614,4.0,1892,2025,134
2184,USW00012841,ORLANDO EXECUTIVE AP,FL,28.5467,-81.3356,31.7,1892,2025,134
1822,USC00080369,AVON PARK 2 W,FL,27.5947,-81.5267,46.9,1892,2022,131


In [9]:
# Search for stations that include the city name provided by the user, and offer the user the option to choose which station to use if multiple matches are found
city_stations = state30_clim[state30_clim['NAME'].str.contains(city, na=False)].copy()

if not city_stations.empty:
    print(f"Found {len(city_stations)} station(s) matching '{city}' in {state}:")
    display(city_stations)
    
    # Ask user to select a station from the list
    stationID = input("Which station would you like to use? Copy and paste the station ID (e.g., 'USW00012839'): ")
    station_name = city_stations.loc[city_stations['ID'] == stationID, 'NAME'].values[0]
    print(f"Using station {city_stations.loc[city_stations['ID'] == stationID, 'NAME'].values[0]} ({stationID})")

else: # if no stations found matching city name, offer user the option to search for the closest station by lat/lon instead
    print( f"No stations found matching '{city}' in {state}.")
    search_option = input("Would you like to search for the closest station by latitude/longitude instead? (yes/no): ")
    if search_option.lower() == 'yes':
        # Get user input for latitude and longitude
        city_lat = float(input("Enter city latitude (e.g., 29.6516): ").strip()) 
        city_lon = float(input("Enter city longitude (e.g., -82.3248): ").strip())

        # Find the closest station by calculating the minimum lat/lon distance from a station to the city coordinates
        state30_clim['DISTANCE'] = np.sqrt((state30_clim['LATITUDE'] - city_lat)**2 + (state30_clim['LONGITUDE'] - city_lon)**2)
        closest_station = state30_clim.loc[state30_clim['DISTANCE'].idxmin()]
        stationID = closest_station['ID']
        station_name = closest_station['NAME']
        print(f"The closest station to this location is {closest_station['NAME']} ({stationID}) at a distance of {closest_station['DISTANCE']:.2f} degrees.")
    else:
        print("Exiting program.")
        exit()



Found 1 station(s) matching 'GAINESVILLE' in FL:


,ID,NAME,STATE,LATITUDE,LONGITUDE,ELEVATION,first,last,years
2171,USW00012816,GAINESVILLE RGNL AP,FL,29.6917,-82.2761,40.5,1960,2025,66


Using station GAINESVILLE RGNL AP (USW00012816)


In [37]:
# Get the data for the station of interest

url = S3_BY_STATION.format(id=stationID)

def load_station_daily(url: str) -> pd.DataFrame:
    df = pd.read_csv(url, storage_options=STOR, dtype={'ID':str,'ELEMENT':str}, parse_dates=['DATE'])
    df['DATA_VALUE'] = df['DATA_VALUE'].replace(-9999, np.nan)
    wide = (df.pivot_table(index=['ID','DATE'], columns='ELEMENT', values='DATA_VALUE', aggfunc='first').reset_index())
    for c in ('TMAX','TMIN','TAVG'):
        if c in wide: wide[c] = wide[c]/10.0
    if 'PRCP' in wide: wide['PRCP'] = wide['PRCP']/10.0
    return wide.sort_values(['ID','DATE']).reset_index(drop=True)

w = load_station_daily(url)
print(f"The GHCN data for {station_name} ({stationID}) has {w.shape} elements")

daily = w.copy()
daily.head()

The GHCN data for GAINESVILLE RGNL AP (USW00012816) has (18545, 46) elements


/var/folders/jc/711xkp256bz0pq8mq4dcp94h0000gn/T/ipykernel_26890/508935664.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, storage_options=STOR, dtype={'ID':str,'ELEMENT':str}, parse_dates=['DATE'])


ELEMENT,ID,DATE,ADPT,ASLP,ASTP,AWBT,AWND,FMTM,MDPR,PGTM,...,WT14,WT15,WT16,WT17,WT18,WT21,WT22,WV01,WV03,WV20
0,USW00012816,1960-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00012816,1960-05-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USW00012816,1960-05-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00012816,1960-05-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USW00012816,1960-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# Format and clean up the data to keep only columns of interest
daily['DATE'] = pd.to_datetime(daily['DATE'])
station_daily= daily[['ID','DATE','PRCP','TMAX','TMIN']].copy()
station_daily

ELEMENT,ID,DATE,PRCP,TMAX,TMIN
0,USW00012816,1960-05-01,NaN,NaN,NaN
1,USW00012816,1960-05-02,NaN,NaN,NaN
2,USW00012816,1960-05-03,NaN,NaN,NaN
3,USW00012816,1960-05-04,NaN,NaN,NaN
4,USW00012816,1960-05-05,NaN,NaN,NaN
...,...,...,...,...,...
18540,USW00012816,2025-02-02,0.0,25.0,5.0
18541,USW00012816,2025-02-03,0.0,24.4,12.2
18542,USW00012816,2025-02-04,0.0,NaN,NaN
18543,USW00012816,2025-02-05,0.0,NaN,NaN


In [39]:
# Write out the station data for future analysis

# Write out as Parquet
station_daily.to_parquet(OUT_PARQUET, index=False)
print('Wrote:', OUT_PARQUET.resolve())
pd.read_parquet(OUT_PARQUET).groupby('ID').size()

# Write out as CSV
station_daily.to_csv(OUT_CSV, index=False)
print('Wrote:', OUT_CSV.resolve())
pd.read_csv(OUT_CSV).groupby('ID').size()

Wrote: /Users/meganwalker/MDW Docs/Academic/Univ Illinois/ATMS 523/module3/data/ghcn_USW00012816.parquet
Wrote: /Users/meganwalker/MDW Docs/Academic/Univ Illinois/ATMS 523/module3/data/ghcn_USW00012816.csv


ID
USW00012816    18545
dtype: int64

### Function that calculates 
1. the all time record high and low and 
2. the normal (mean) high and low temperature 
FOR EACH CALENDAR DAY for the 1991-2020 period for the desired station. 

The function returns a pandas data frame with the columns ['record_min_temp', average_min_temp', 'average_max_temp', record_max_temp'] FOR EACH DAY. 

In [40]:
# Gather data for only 1991-2020 period
station_daily['DATE'] = pd.to_datetime(station_daily['DATE'])
climate_period = (station_daily['DATE'].dt.year >= 1991) & (station_daily['DATE'].dt.year <= 2020)
climate_data = station_daily[climate_period].copy()

# Create a day-of-year column to facilitate calculation of daily climatologies
climate_data['dayofyear'] = climate_data['DATE'].dt.dayofyear

# Group by day of year 
DOY_grouped = climate_data.groupby('dayofyear')



In [44]:
# Calculate the daily climatology for max_TMAX, min_TMIN, avg_TMAX, and avg_TMIN
climatology = DOY_grouped.agg(
    record_max_temp = ('TMAX', 'max'),
    record_min_temp = ('TMIN', 'min'),
    average_max_temp = ('TMAX', 'mean'),
    average_min_temp = ('TMIN', 'mean')
).reset_index()

# Get rid of Leap Day (Feb 29) artifacts for simplicity 
# dayofyear 60 is Feb 29 in leap years
# dayofyear 366 is Dec 31 in leap years, which rolls over to Jan 1 in non-leap years
climatology = climatology[(climatology['dayofyear'] != 60) & (climatology['dayofyear'] != 366)]

# Create column that displays the month and day for each day of the year for prettier plotting later
climatology['month_day'] = pd.to_datetime(climatology['dayofyear'], format='%j').dt.strftime('%m-%d')

climatology

,dayofyear,record_max_temp,record_min_temp,average_max_temp,average_min_temp,month_day
0,1,27.8,-5.6,20.906667,8.396667,01-01
1,2,29.4,-3.9,20.483333,9.100000,01-02
2,3,28.9,-5.6,18.456667,7.256667,01-03
3,4,27.2,-6.7,18.480000,6.143333,01-04
4,5,27.2,-6.1,18.890000,5.426667,01-05
...,...,...,...,...,...,...
360,361,28.9,-4.3,19.166667,5.950000,12-27
361,362,29.4,-6.1,20.036667,6.543333,12-28
362,363,27.2,-5.0,20.886667,7.830000,12-29
363,364,28.9,-0.6,20.376667,6.840000,12-30


In [45]:
# Logic developed in the previous two code cells - now createing a function to encapsulate it

def calculate_station_climatology(csv_path):
    """
    Calculate daily climatology for a station from a CSV file.
    The CSV must contain columns: 'ID', 'DATE', 'TMAX', 'TMIN', as found in GHCN data.
    Returns a DataFrame with columns:
    ['dayofyear', 'record_max_temp', 'record_min_temp', 'average_max_temp', 'average_min_temp', 'month_day']
    """
    import pandas as pd
    # Load data
    df = pd.read_csv(csv_path)
    df['DATE'] = pd.to_datetime(df['DATE'])
    # Filter for 1991-2020
    climate_period = (df['DATE'].dt.year >= 1991) & (df['DATE'].dt.year <= 2020)
    climate_data = df[climate_period].copy()
    # Create dayofyear column
    climate_data['dayofyear'] = climate_data['DATE'].dt.dayofyear
    # Remove leap day
    climate_data = climate_data[climate_data['dayofyear'] != 60].copy()
    # Group by day of year
    DOY_grouped = climate_data.groupby('dayofyear')
    # Calculate climatology
    climatology = DOY_grouped.agg(
        record_max_temp = ('TMAX', 'max'),
        record_min_temp = ('TMIN', 'min'),
        average_max_temp = ('TMAX', 'mean'),
        average_min_temp = ('TMIN', 'mean')
    ).reset_index()
    # Remove any dayofyear > 365 
    climatology = climatology[climatology['dayofyear'] <= 365].copy()
    # Add month_day column
    climatology['month_day'] = pd.to_datetime(climatology['dayofyear'], format='%j').dt.strftime('%m-%d')
    return climatology


In [46]:
# Demonstrate the function works by applying it to the station data we have been working with
csv_path = OUT_CSV
calculate_station_climatology(csv_path)



,dayofyear,record_max_temp,record_min_temp,average_max_temp,average_min_temp,month_day
0,1,27.8,-5.6,20.906667,8.396667,01-01
1,2,29.4,-3.9,20.483333,9.100000,01-02
2,3,28.9,-5.6,18.456667,7.256667,01-03
3,4,27.2,-6.7,18.480000,6.143333,01-04
4,5,27.2,-6.1,18.890000,5.426667,01-05
...,...,...,...,...,...,...
359,361,28.9,-4.3,19.166667,5.950000,12-27
360,362,29.4,-6.1,20.036667,6.543333,12-28
361,363,27.2,-5.0,20.886667,7.830000,12-29
362,364,28.9,-0.6,20.376667,6.840000,12-30


pandas._libs.tslibs.timestamps.Timestamp